In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'f:\\Project_MultitaskModel'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationModelConfig:
    trained_model_path: Path
    data_classification: Path
    data_segmentation: Path
    repo_name: str
    repo_owner: str
    batch_size: int
    n_classes: int
    n_segment: int
    in_channels: int
    num_workers: int
    img_size: int
    seed: int
    augmentation: bool
    all_params: dict
    
    

In [4]:
from src.Project_MultitaskModel.constants import *
from src.Project_MultitaskModel.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigureManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
    def get_evaluation_model_config(self) -> EvaluationModelConfig:
        training = self.config.training_model
        trained_model_path = Path(training.trained_model_path)
        mlflow_tracking_uri = os.getenv('MLFLOW_TRACKING_REPO')
        repo_owner = os.getenv('MLFLOW_TRACKING_USERNAME')

        evaluation_model_config = EvaluationModelConfig(
            trained_model_path=trained_model_path,
            data_classification=Path(training.data_classification),
            data_segmentation=Path(training.data_segmentation),
            repo_name=mlflow_tracking_uri,
            repo_owner=repo_owner,
            batch_size=self.params.BATCH_SIZE,
            num_workers=self.params.NUM_WORKERS,
            img_size=self.params.IMAGE_SIZE,
            seed=self.params.SEED,
            augmentation=False,
            all_params=self.params,
            n_classes=self.params.N_CLASSES,
            n_segment=self.params.N_SEGMENT,
            in_channels=self.params.IN_CHANNELS
        )
        return evaluation_model_config

In [6]:
import torch
from src.models.multi_task_model import MultiTaskModelResNet
from src.data.loader_data import data_loader
from src.metrics import calculate_dice, calculate_iou
import dagshub
import mlflow
import mlflow.pytorch
from urllib.parse import urlparse
from pathlib import Path

In [7]:
class Evaluation:
    def __init__(self, config: EvaluationModelConfig):
        self.config = config
    
    def loader_data(self):

        test_class_path = os.path.join(self.config.data_classification, 'test')
        test_seg_path = os.path.join(self.config.data_segmentation, 'test')

        test_loader = data_loader(
            data_classification_path=test_class_path,
            data_segmentation_path=test_seg_path,
            batch_size=self.config.batch_size,
            shuffle=False,
            num_workers=self.config.num_workers,
            augmentation=self.config.augmentation,
            seed=self.config.seed,
            img_size=self.config.img_size
        )

        return test_loader

    def load_model(self):
        model = MultiTaskModelResNet(
            n_classes=self.config.n_classes,
            n_segment=self.config.n_segment,
            in_channels=self.config.in_channels,
            pretrained=False
        )
        state_dict = torch.load(self.config.trained_model_path, map_location=torch.device('cpu'))
        model.load_state_dict(state_dict)
        return model

    def evaluation(self):
        self.model = self.load_model()
        test_loader = self.loader_data()
        
        
        self.model.eval()
        with torch.no_grad():
            total_dice = 0.0
            total_iou = 0.0
            correct = 0

            for i, (images, masks, labels) in enumerate(test_loader):
                outputs_class, outputs_seg = self.model(images)

                _, predicted = torch.max(outputs_class, 1)
                correct += (predicted == labels).sum().item()
                dice = calculate_dice(outputs_seg, masks)
                iou = calculate_iou(outputs_seg, masks)
                total_dice += dice.item()
                total_iou += iou.item()
            accuracy = 100 * correct / len(test_loader.dataset)
            avg_dice = total_dice / len(test_loader)
            avg_iou = total_iou / len(test_loader)
        self.scores = {'accuracy': accuracy, 'dice': avg_dice, 'iou': avg_iou}
        
        self.save_score()

    def save_score(self):
        save_json(path=Path("scores.json"), data=self.scores)
    
    def log_into_mlflow(self):
        dagshub.init(repo_owner=str(self.config.repo_owner), repo_name=str(self.config.repo_name))
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.pytorch.log_model(self.model, name="model")
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(self.scores)
            if tracking_url_type_store != "file":
                # Vừa lưu mô hình, vừa ĐĂNG KÝ tên mô hình lên Registry
                mlflow.pytorch.log_model(
                    self.model, 
                    "model", 
                    registered_model_name="MultiTaskModelResNet"
                )
            else:
                # Nếu là file cục bộ, chỉ lưu file mô hình thôi (không đăng ký version)
                mlflow.pytorch.log_model(self.model, "model")
        

In [8]:
try:
    config = ConfigureManager()
    eval_config = config.get_evaluation_model_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-12-20 00:49:54,803: INFO: common: yaml file: configs\config.yaml loaded successfully]
[2025-12-20 00:49:54,806: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-20 00:55:18,837: INFO: common: json file saved at: scores.json]
[2025-12-20 00:55:20,500: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as QuangTruong2612

[2025-12-20 00:55:20,513: INFO: helpers: Accessing as QuangTruong2612]
[2025-12-20 00:55:21,721: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/QuangTruong2612/Project_MultitaskModel "HTTP/1.1 200 OK"]
[2025-12-20 00:55:23,908: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "QuangTruong2612/Project_MultitaskModel"

[2025-12-20 00:55:23,913: INFO: helpers: Initialized MLflow to track repo "QuangTruong2612/Project_MultitaskModel"]


Repository QuangTruong2612/Project_MultitaskModel initialized!

[2025-12-20 00:55:23,915: INFO: helpers: Repository QuangTruong2612/Project_MultitaskModel initialized!]
🏃 View run unequaled-colt-233 at: https://dagshub.com/QuangTruong2612/Project_MultitaskModel.mlflow/#/experiments/0/runs/eff2bc4ffb474ec497aadc08138659b7
🧪 View experiment at: https://dagshub.com/QuangTruong2612/Project_MultitaskModel.mlflow/#/experiments/0


KeyboardInterrupt: 